In [1]:
#在tensorboard上查看数据的变化
#cmd tensorboard --logdir=D:\Anaconda3\Tensorflow\logs
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("D:/Anaconda3/Tensorflow/MNIST_data",one_hot=True)
batch_size = 100
n_batch = mnist.train.num_examples // batch_size

#参数概要，分析所记录值的mean stddev max min
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        #记录该值，并给这个值名字
        tf.summary.scalar('mean',mean)#平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev',stddev)#标准差
        tf.summary.scalar('max',tf.reduce_max(var))#最大值
        tf.summary.scalar('min',tf.reduce_min(var))#最小值
        tf.summary.histogram('histogram',var)#直方图
#命名空间
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32,[None,784],name='x-input')
    y = tf.placeholder(tf.float32,[None,10],name='y-input')

with tf.name_scope('layer'):
    #784-10无隐藏层
    with tf.name_scope('wights'):
        W = tf.Variable(tf.zeros([784,10]),name='W')
        #使用定义的函数，把这个值传进去，记录并分析
        variable_summaries(W)
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([10]),name='b')
        variable_summaries(b)
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(x,W) + b
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(wx_plus_b)

with tf.name_scope('loss'):
    #loss =tf.reduce_mean(tf.square(y-prediction))
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    #没有调用上面定义的函数，因为loss数据只有一个，无需计算平均值，标准差
    tf.summary.scalar('loss',loss)
with tf.name_scope('train'):
    #train = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
    train = tf.train.AdamOptimizer(1e-2).minimize(loss)

init = tf.global_variables_initializer()

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)
        
#合并所有的summary
#因为监测的指标很多
merged = tf.summary.merge_all()
        
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('D:/Anaconda3/Tensorflow/logs/',sess.graph)
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            #同时运行merge train 一边训练一边统计
            #run完后会有反馈 merge的反馈值存在summary中
            summary,_ = sess.run([merged,train],feed_dict={x:batch_xs,y:batch_ys})
        #记录summary 和 epoch  写到文件中   
        writer.add_summary(summary,epoch)
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter " + str(epoch) + ",Testing Accuracy " + str(acc))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting D:/Anaconda3/Tensorflow/MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting D:/Anaconda3/Tensorflow/MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting D:/Anaconda3/Tensorflow/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting D:/Anaconda3/Tensorflow/MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Iter 0,Testing

In [3]:
#关于tf.summary.scalar的学习在以下URL有说明
#https://blog.csdn.net/ls617386/article/details/62049834